In [49]:
import pandas as pd
from bs4 import BeautifulSoup
import requests
import json 

In [50]:
dfRem=pd.read_excel("Remdesivir.xlsx",skiprows=[0])
#dfRem=dfRem[:10]

In [51]:
dfRem.head(4)

,Rank,Publication ID,DOI,PMID,PMCID,Title,Source title,Anthology title,Publisher,MeSH terms,...,Source Linkout,Dimensions URL,FOR (ANZSRC) Categories,RCDC Categories,HRCS HC Categories,HRCS RAC Categories,ICRP Cancer Types,ICRP CSO Categories,Units of Assessment,Sustainable Development Goals
0,496,pub.1127401342,10.1186/s40779-020-00251-x,32370766.0,PMC7199873,Role of adjunctive treatment strategies in COV...,Military Medical Research,NaN,NaN,Betacoronavirus; Coronavirus Infections; Human...,...,https://mmrjournal.biomedcentral.com/track/pdf...,https://app.dimensions.ai/details/publication/...,11 Medical and Health Sciences; 1103 Clinical ...,Rare Diseases,NaN,6.1 Pharmaceuticals,NaN,NaN,"A03 Allied Health Professions, Dentistry, Nurs...",NaN
1,429,pub.1127456489,10.1186/s13037-020-00245-7,32395179.0,PMC7206578,Successful COVID-19 rescue therapy by extra-co...,Patient Safety in Surgery,NaN,Springer Nature,NaN,...,https://pssjournal.biomedcentral.com/track/pdf...,https://app.dimensions.ai/details/publication/...,11 Medical and Health Sciences; 1103 Clinical ...,Pneumonia; Lung; Prevention; Clinical Research,NaN,NaN,NaN,NaN,A01 Clinical Medicine,3 Good Health and Well Being
2,360,pub.1126815092,10.1186/s13613-020-00661-z,32307593.0,PMC7167303,Therapeutic strategies for critically ill pati...,Annals of Intensive Care,NaN,Springer Nature,NaN,...,https://annalsofintensivecare.springeropen.com...,https://app.dimensions.ai/details/publication/...,11 Medical and Health Sciences; 1103 Clinical ...,Infectious Diseases; Lung; Prevention; Influen...,Infection,NaN,NaN,NaN,"A02 Public Health, Health Services and Primary...",NaN
3,348,pub.1127536759,10.1186/s13046-020-01590-2,32398164.0,PMC7214852,Drug repurposing against COVID-19: focus on an...,Journal of Experimental & Clinical Cancer Rese...,NaN,Springer Nature,Antineoplastic Agents; Antiviral Agents; Betac...,...,https://jeccr.biomedcentral.com/track/pdf/10.1...,https://app.dimensions.ai/details/publication/...,11 Medical and Health Sciences; 1115 Pharmacol...,Cancer; Vaccine Related; Infectious Diseases; ...,Cancer; Infection,5.1 Pharmaceuticals,Not Site-Specific Cancer,5.3 Systemic Therapies - Discovery and Develop...,"A03 Allied Health Professions, Dentistry, Nurs...",NaN


In [52]:
len(dfRem)

1818

# Webseiten auswerten

In [53]:
pubkeys=[
    'title',
    'aff_org_name',
    'researcher_dim_id',
     'researcher_dim_count',
     'journal_title',
     'language',
     'abstract',
     'open_access',
     'publisher',
     'aff_city_name',
     'aff_country_name',
     'doi',
     'pub_date',
     'pub_year',
     'times_cited',
     'altmetric_id',
     'altmetric',
     'authors_full',
        ]

In [54]:
def getmeta(x):
    html=""
    d=""
    v=""
    url=x["Dimensions URL"]
    if url!=None:
        html = requests.get(url).content
    soup=BeautifulSoup(html,"lxml")
    datadoc=soup.find("div")
    if datadoc==None:
        attr=""
    else:
        attr=datadoc.get("data-doc")
        if attr:
            d=json.loads(attr)
    dicpub=dict()
    for i in pubkeys:
        if d!="":
            v=d.get(i,"NaN")
        dicpub[i]=v
    return(dicpub)

In [55]:
dfRem["seldict"]=dfRem.apply(getmeta,axis=1)

In [56]:
lidi=[]
def newpub(x):
    di=dict()
    d=x["seldict"]
    di["DOI"]=x["DOI"]
    di["title"]=x["Title"]
    di["authors"]=d["authors_full"]
    di["publisher"]=d["publisher"]
    di["source"]=x["Source title"]
    di["aff_org_name"]=d["aff_org_name"]
    di["aff_country"]=d["aff_country_name"]
    di["pub_date"]=d["pub_date"]
    di["abstract"]=d["abstract"]
    di["openaccess"]=d["open_access"]
    di["di_URL"]=x["Dimensions URL"]
    lidi.append(di)

In [57]:
dfRem.apply(lambda x:newpub(x),axis=1)
dfRemLit=pd.DataFrame(lidi)
dfRemLit.head(3)

,DOI,title,authors,publisher,source,aff_org_name,aff_country,pub_date,abstract,openaccess,di_URL
0,10.1186/s40779-020-00251-x,Role of adjunctive treatment strategies in COV...,"[Xinni Xu, Yew Kwang Ong, De Yun Wang]",NaN,Military Medical Research,"[National University of Singapore, Department ...",[Singapore],2020-12,The coronavirus disease (COVID-19) pandemic ha...,True,https://app.dimensions.ai/details/publication/...
1,10.1186/s13037-020-00245-7,Successful COVID-19 rescue therapy by extra-co...,"[Michael S. Firstenberg, Philip F. Stahel, Jen...",Springer Nature,Patient Safety in Surgery,"[The Medical Center of Aurora, Rocky Vista Uni...",[United States],2020-12,Background: The value of extracorporeal membra...,True,https://app.dimensions.ai/details/publication/...
2,10.1186/s13613-020-00661-z,Therapeutic strategies for critically ill pati...,"[Lei Li, Ranran Li, Zhixiong Wu, Xianghong Yan...",Springer Nature,Annals of Intensive Care,"[Huadong Hospital, Shanghai Jiao Tong Universi...",[China],2020-12,Since the 2019 novel coronavirus disease (COVI...,True,https://app.dimensions.ai/details/publication/...


In [58]:
dfRemLit.to_json("dfRemLit.json")

In [59]:
len(dfRemLit)

1818